<a href="https://colab.research.google.com/github/honi-sm/AmphiBIO_TraitAnalysisFinal/blob/main/00CleanedDataPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy openpyxl


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving AmphiBIOv1Raw.xlsx to AmphiBIOv1Raw.xlsx


In [ ]:
"""
AmphiBIO Data Cleaning
Author: Lia Grace Stratos
Institution: University of Nebraska Omaha
Date: November 2025

Purpose:
    Reproduces what Idid in Excel for the AmphiBIO dataset.
    It filters out incomplete or invalid entries, standardizes text and numeric
    and combines the 0 or 1 columns into categorical variables for habitat type and
    reproductive method. Then gets a seed to reproduce random sample of 500 species.

Use:
    Run this script in the same directory as AmphiBIOv1Raw.xlsx
    The output includes two CSV files:
        1. cleanDataPython.csv
        2. randomSample_500_python.csv

Expected:
    - Original dataset had 6,777 rows
    - After cleaning is about 1,481 usable species
    - All missing values ('NA', blanks, zeros) are treated consistently
"""

import pandas as pd
import numpy as np
import sys
import os

# File path
RAW_FILE = "AmphiBIOv1Raw.xlsx"
OUTPUT_CLEAN = "cleanDataPython.csv"
OUTPUT_SAMPLE = "randomSample_500_python.csv"
RANDOM_SEED = 2025


# Load dtat
def load_data(file_path: str) -> pd.DataFrame:
    """Load the raw dataset with missing value handling."""
    if not os.path.exists(file_path):
        sys.exit(f"Error: File '{file_path}' not found lol, need glasses?")
    try:
        df = pd.read_excel(
            file_path,
            engine="openpyxl",
            na_values=["NA", "N/A", "na", "NaN", "", " "]
        )
        print(f"Loaded file: {file_path} ({df.shape[0]} rows, {df.shape[1]} columns)")
        return df
    except Exception as e:
        sys.exit(f"I can't reaaaaaad '{file_path}': {e}")


# Check columns
def validate_columns(df: pd.DataFrame, required: list[str]) -> None:
    """Verify all necessary columns are present."""
    missing = [c for c in required if c not in df.columns]
    if missing:
        sys.exit(f"Error: Missing required columns: {missing}")


# trandlate the raw habitat types
def make_habitat(row: pd.Series) -> str:
    parts = []
    if row["Fos"] == 1: parts.append("fossorial")
    if row["Ter"] == 1: parts.append("terrestrial")
    if row["Aqu"] == 1: parts.append("aquatic")
    if row["Arb"] == 1: parts.append("arboreal")
    return " + ".join(parts) if parts else np.nan


# translate reproductiveMethod
def make_reproduction(row: pd.Series) -> str:
    if row["Dir"] == 1:
        return "direct"
    elif row["Lar"] == 1:
        return "larval"
    elif row["Viv"] == 1:
        return "viviparous"
    else:
        return np.nan


# export that biatch
def safe_export(df: pd.DataFrame, filename: str) -> None:
    """Write DataFrame to CSV with error handling."""
    try:
        df.to_csv(filename, index=False)
        print(f"Saved: {filename} ({len(df)} rows)")
    except PermissionError:
        print(f"Error: Permission denied while saving '{filename}'. Close any open file and retry.")
    except Exception as e:
        print(f"Unexpected error saving '{filename}': {e}")


# Loaading raw for the CSV
raw = load_data(RAW_FILE)

# Drop metadata but make note for the other sheet
drop_cols = ["id", "Order", "Family", "Genus", "OBS"]
raw = raw.drop(columns=[c for c in drop_cols if c in raw.columns])

# Keep other columns
cols = [
    "Species", "Fos", "Ter", "Aqu", "Arb",
    "Body_size_mm", "Litter_size_min_n", "Litter_size_max_n",
    "Dir", "Lar", "Viv"
]
validate_columns(raw, cols)
data = raw[cols].copy()

# Normalize text
data["Species"] = (
    data["Species"]
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
)

# Type casting fields :p
num_cols = ["Body_size_mm", "Litter_size_min_n", "Litter_size_max_n"]
bin_cols = ["Fos", "Ter", "Aqu", "Arb", "Dir", "Lar", "Viv"]

for col in num_cols:
    try:
        data[col] = pd.to_numeric(data[col], errors="coerce")
    except Exception as e:
        print(f"Warning: could not convert {col} to numeric: {e}")

for col in bin_cols:
    data[col] = pd.to_numeric(data[col], errors="coerce").astype("float64")

# Replace zeros with NaN (Excel was treating them as blanks, messing up cleaning)
data.replace(0, np.nan, inplace=True)

# Drop rows with NaN
before = len(data)
data = data[
    data["Body_size_mm"].notna() &
    data["Litter_size_min_n"].notna() &
    data["Litter_size_max_n"].notna()
]
after = len(data)
print(f"Removed {before - after} rows with missing or invalid numeric data.")

data["habitatType"] = data.apply(make_habitat, axis=1)
data["reproductiveMethod"] = data.apply(make_reproduction, axis=1)

# Drop missing reproductive data aka NaN
data = data[data["reproductiveMethod"].notna()]

# Drop duplicate rows
data["Species"] = data["Species"].str.strip().str.lower()
before_dupes = len(data)
data = data.drop_duplicates(subset="Species", keep="first")
print(f"Removed {before_dupes - len(data)} duplicate species entries.")

# Rename and reorder columns to prefered, I like camel case because it looks nicer lowk
data = data.rename(columns={
    "Body_size_mm": "bodyLengthMm",
    "Litter_size_min_n": "clutchSizeMinN",
    "Litter_size_max_n": "clutchSizeMaxN"
})
cleaned = data[[
    "Species", "bodyLengthMm", "clutchSizeMinN", "clutchSizeMaxN",
    "habitatType", "reproductiveMethod"
]].reset_index(drop=True)

# ALL CLEAAAAAN yiPPEEEE
safe_export(cleaned, OUTPUT_CLEAN)

# OK random samp time
try:
    sample = cleaned.sample(n=500, random_state=RANDOM_SEED)
    safe_export(sample, OUTPUT_SAMPLE)
except ValueError as e:
    print(f"Error creating random sample: {e}")

print(f"Checked that: {len(cleaned)} species (expected ≈1481).")
print("Squeaky Clean Sir.")

Loaded file: AmphiBIOv1Raw.xlsx (6776 rows, 38 columns)
Removed 5181 rows with missing or invalid numeric data.
Removed 0 duplicate species entries.
Saved: cleanDataPython.csv (1484 rows)
Saved: randomSample_500_python.csv (500 rows)
Checked that: 1484 species (expected ≈1481).
Squeaky Clean Sir.


In [ ]:
!ls /content

AmphiBIOv1Raw.xlsx   randomSample_500_python.csv
cleanDataPython.csv  sample_data


In [ ]:
"""
Creates an expanded habitat column, needed for charts.
ex "fossorial + terrestrial" -> two rows: fossorial, terrestrial
"""

import pandas as pd
import numpy as np

# Split habitat strings
hab_split = (
    data["habitatType"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "")
    .str.split(r"\+|,|;|/")
)

# Explode into rows
data_expanded = data.copy()
data_expanded = data_expanded.assign(habitat_split=hab_split).explode("habitat_split")

# Remove empty or NA entries
data_expanded = data_expanded.dropna(subset=["habitat_split"])
data_expanded["habitat_split"] = data_expanded["habitat_split"].str.strip()
data_expanded = data_expanded[data_expanded["habitat_split"] != ""]

# Standardize
replace_map = {
    "fossorial": "fossorial",
    "fosorial": "fossorial",
    "fossforial": "fossorial",
    "terrestrial": "terrestrial",
    "aquatic": "aquatic",
    "arboreal": "arboreal"
}

data_expanded["habitat_split"] = data_expanded["habitat_split"].replace(replace_map)

# Final categorical list
valid_habitats = ["fossorial", "terrestrial", "aquatic", "arboreal"]
data_expanded = data_expanded[data_expanded["habitat_split"].isin(valid_habitats)]

print("Counts of habitat (one row per habitat per species):")
display(data_expanded["habitat_split"].value_counts())


Counts of habitat (one row per habitat per species):


,count
habitat_split,
terrestrial,1360
aquatic,1060
arboreal,646
fossorial,195


In [ ]:
# same 500 sample (seed = 2025)
RANDOM_SEED = 2025
sample500 = cleaned.sample(n=500, random_state=RANDOM_SEED).copy()

# Apply log10
numeric_cols = ["bodyLengthMm", "clutchSizeMinN", "clutchSizeMaxN"]

for col in numeric_cols:
    sample500[f"log_{col}"] = np.log10(sample500[col])

# Save to processed folder
output_path = "sample500_with_logs.csv"
sample500.to_csv(output_path, index=False)

print(f"Saved: {output_path} ({len(sample500)} rows)")
print("Columns: log_bodyLengthMm, log_clutchSizeMinN, log_clutchSizeMaxN")

NameError: name 'cleaned' is not defined